<a href="https://colab.research.google.com/github/AnastasiyaKS/IDA-labs-RU/blob/master/IDA_Lab_1_0_Data_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://www.bigdatauniversity.com"><img src = "https://ibm.box.com/shared/static/jvcqp2iy2jlx2b32rmzdt0tx8lvxgzkp.png" width = 300, align = "center"></a>

#<center> Анализ данных с использованием Python </center>


<hr>

##<center>Выбросы CO2 по странам по годам </center>

Выбросы углекислого газа - это выбросы от сжигания ископаемого топлива и производства цемента. Они включают углекислый газ, образующийся при потреблении твердого, жидкого и газового топлива и сжигания газа.

http://data.worldbank.org/indicator/EN.ATM.CO2E.PC/

### Получение данных

Данные можно загрузить по ссылке The World Bank [here](http://data.worldbank.org/indicator/EN.ATM.CO2E.PC/) либо [here](https://ibm.box.com/shared/static/3yzxbbizo49bkl8cnjw15tymzfwkycj4.csv)

#### Здесь мы будем использовать команду ***wget***, чтобы получить файл ***csv*** из прямой ссылки

In [0]:
!wget --output-document co2emissions.csv https://ibm.box.com/shared/static/3yzxbbizo49bkl8cnjw15tymzfwkycj4.csv

<hr>

### Импорт данныx с помощью Pandas

#### Импорт `pandas` library

In [0]:
import pandas as pd

#### Импорт данных с помощью `pd.read_csv`

In [0]:
data = pd.read_csv("co2emissions.csv", skiprows = 4)

#### Отобразим первые 5 строк  `data`, используя ` head`

In [0]:
data.head()

####Очистка данных</h2> 

Посмотрите на данные. Какие проблемы есть с качеством данных и как их решить?

Например, что не так с этими строками?

In [0]:
data.loc[[38,93,151,174,242]]

#### Проблемы с качеством данных:
1. Некоторые строки представляют собой совокупности стран, а не фактические страны (например, «Мир»).
2. Некоторые столбцы не имеют значения и могут быть удалены (например, «Имя индикатора»).
3. Некоторые годы не имеют данных для какой-либо страны (например, с 2012 по 2015 год).
4. В некоторых странах нет данных за год. (например, «Тайвань, Китай»)

## Устраним найденные недостатки

### 1. Некоторые строки представляют собой совокупности стран, а не фактические страны (например, «Мир»). 

** Цели: **
Удалите строки, которые не содержат фактическую страну. К счастью, Всемирный банк предоставляет нам метаданные, по которым строки являются странами и являются агрегатами.
- import countries_metadata.csv
- объединим (Merge) метаданные с `data`  по ` Country Code`

#### Загрузим `countries_metadata.csv`

In [0]:
!wget --output-document countries_metadata.csv https://ibm.box.com/shared/static/qh3o86mpij17ot7anydcmbwt41lwxvln.csv

#### Импортируем `countries_metadata.csv`

In [0]:
metadata = pd.read_csv("countries_metadata.csv", encoding = "utf-8")

In [0]:
metadata.head(10)

#### Как определить, когда указанное «Country Name» является названием страны, а когда - агрегированным регионом?

Можно заметить,  когда **строка** представляет собой **совокупность стран**, например, «Арабский мир» --> **`Region`** и **`IncomeGroup`** представляют собой **NaN** (Not a Number). Мы можем использовать это правило для удаления всех регионов, которые не являются странами.

#### Объединить `данные` с` метаданными` по ключу `Код страны`

**Merge** `data` with `metadata` on the key, `Country Code`

In [0]:
merge = pd.merge(data, metadata, on = "Country Code")

In [0]:
merge.head(10)

** Примечание: ** Значения региона являются NaN, когда строка не является реальной страной.

#### Удалите строки, где `Region` является NaN

In [0]:
merge = merge[pd.notnull(merge['Region'])]

In [0]:
merge.head(10)

### 2. Некоторые столбцы не имеют значения и могут быть удалены.

** Цели: **

Удалите следующие неуместные столбцы:
- Столбец 3: ** «Имя индикатора» **
- Столбец 4: ** «Код индикатора» **

In [0]:
merge.columns

In [0]:
# справочную информацию о функции можно получить, добавив к названию функции вопросительный знак
merge.drop?

In [0]:
merge = merge.drop(merge.columns[[60,61,64,65]], axis=1) # Note: zero indexed
merge = merge.drop('Indicator Name', axis=1)
merge = merge.drop('Indicator Code', 1)

In [0]:
merge.columns

### 3. Некоторые годы не содержат данных ни для одной страны



** Цели: **
Подсчитайте количество строк для каждого года. Значение NaN не суммируется к общей сумме.


In [0]:
merge.count()

Проверим 2015 год, поскольку, похоже, для этого года нет строк, содержащих данные.

In [0]:
merge['2015']

#### Удалим столбцы, которые не содержат данные

In [0]:
merge = merge.drop(['2012', '2013', '2014', '2015'], 1)

In [0]:
merge.count() #double-check that columns have been removed

### 4. В некоторых странах нет данных за год.

**Цели:**
Используйте значения строк, чтобы определить, для каких стран нет данных.

Посмотрим среднее значение всех данных для каждой строки (по оси 1).

In [0]:
merge.mean(axis=1) #Takes the mean of all numeric quantities by row

Как вы можете видеть, NaN появляется, когда нет данных для этой строки.

#### Удалим строки, в которых нет данных ни за один год

In [0]:
merge = merge[pd.notnull(merge.mean(axis=1))]

## <center>Data Cleaning ... done!</center>

### Переименуем полученный результат

In [0]:
mydf = merge

<hr>

### А теперь обзор всех выполненных действий с данными.

Напомним:

In [0]:
import pandas as pd

#Download data
#!wget --output-document co2emissions.csv https://ibm.box.com/shared/static/3yzxbbizo49bkl8cnjw15tymzfwkycj4.csv
#!wget --output-document countries_metadata.csv https://ibm.box.com/shared/static/qh3o86mpij17ot7anydcmbwt41lwxvln.csv
    
#Import data
data = pd.read_csv("/resources/data/co2emissions.csv", skiprows = 4)
metadata = pd.read_csv("/resources/data/countries_metadata.csv", encoding = "utf-8")

#Merge data
merge = pd.merge(data, metadata, on = "Country Code")

#Remove non-country regions
merge = merge[pd.notnull(merge['Region'])]

#Drop some columns with no data
merge = merge.drop(merge.columns[[60,61,64,65]], axis=1)
merge = merge.drop(['Indicator Name', 'Indicator Code','2012', '2013', '2014', '2015'], 1)

#Drop some rows with no data
merge = merge[pd.notnull(merge.mean(axis=1))]

#Rename
mydf = merge

In [0]:
mydf

## Экспортируем очищенные данные в файл csv

In [0]:
mydf.to_csv("co2emissions_cleaned.csv", index = False) #See Recent Data for exported csv

## Want to learn more?

You can take free [Python 101](https://cocl.us/DX0108EN_PY0101EN) or [Data Analysis with Python](https://cocl.us/DX0108EN_DA0101EN) courses.  

Also, you can use Data Science Experience to run these notebooks faster with bigger datasets. Data Science Experience is IBM's leading cloud solution for data scientists, built by data scientists. With Jupyter notebooks, RStudio, Apache Spark and popular libraries pre-packaged in the cloud, DSX enables data scientists to collaborate on their projects without having to install anything. Join the fast-growing community of DSX users today with a free account at [Data Science Experience](https://cocl.us/DX0108EN_DSX). 

<hr>
Copyright &copy; 2017 [Cognitive Class](https://cognitiveclass.ai/?utm_source=cccopyrightlink&utm_medium=cclabs). This notebook and its source code are released under the terms of the [MIT License](https://cognitiveclass.ai/mit-license/).


<h3>Authors:</h3>
<article class="teacher">
<div class="teacher-image" style="    float: left;
    width: 115px;
    height: 115px;
    margin-right: 10px;
    margin-bottom: 10px;
    border: 1px solid #CCC;
    padding: 3px;
    border-radius: 3px;
    text-align: center;"><img class="alignnone wp-image-2258 " src="https://ibm.box.com/shared/static/tyd41rlrnmfrrk78jx521eb73fljwvv0.jpg" alt="Saeed Aghabozorgi" width="178" height="178" /></div>
<h4>Saeed Aghabozorgi</h4>
<p><a href="https://ca.linkedin.com/in/saeedaghabozorgi">Saeed Aghabozorgi</a>, PhD is a Data Scientist in IBM with a track record of developing enterprise level applications that substantially increases clients’ ability to turn data into actionable knowledge. He is a researcher in data mining field and expert in developing advanced analytic methods like machine learning and statistical modelling on large datasets.</p>
</article>
<br>
<article class="teacher">
<div class="teacher-image" style="    float: left;
    width: 115px;
    height: 115px;
    margin-right: 10px;
    margin-bottom: 10px;
    border: 1px solid #CCC;
    padding: 3px;
    border-radius: 3px;
    text-align: center;"><img class="alignnone size-medium wp-image-2177" src="https://ibm.box.com/shared/static/2ygdi03ahcr97df2ofrr6cf8knq4kodd.jpg" alt="Polong Lin" width="300" height="300" /></div>
<h4>Polong Lin</h4>
<p>
<a href="https://ca.linkedin.com/in/polonglin">Polong Lin</a> is a Data Scientist and Lead Data Science Advocate at IBM. Polong is a regular speaker in conferences and meetups where he teaches data science. Polong holds a M.Sc. in Cognitive Psychology.</p>
</article>